In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/gdrive', force_remount=True);
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from PIL import Image  #use this or cv2 to crop 


train_images = []
train_labels = []
print('Loading train images...')
for img in glob.glob(r'/gdrive/My Drive/CoLab/TrashTraining/cardboard/*.jpg'): 
  train_images.append(Image.open(img));
  #train_images.append(cv2.imread(img));
  train_labels.append(0)
print('Loaded carboard images...')
for img in glob.glob(r'/gdrive/My Drive/CoLab/TrashTraining/glass/*.jpg'): 
  train_images.append(Image.open(img));
  #train_images.append(cv2.imread(img));
  train_labels.append(1)
print('Loaded glass images...')
for img in glob.glob(r'/gdrive/My Drive/CoLab/TrashTraining/metal/*.jpg'): 
  train_images.append(Image.open(img));
  #train_images.append(cv2.imread(img));
  train_labels.append(2)
print('Loaded metal images...')
for img in glob.glob(r'/gdrive/My Drive/CoLab/TrashTraining/paper/*.jpg'):
  train_images.append(Image.open(img));
  #train_images.append(cv2.imread(img));
  train_labels.append(3)
print('Loaded paper images...')
for img in glob.glob(r'/gdrive/My Drive/CoLab/TrashTraining/plastic/*.jpg'):
  train_images.append(Image.open(img));
  #train_images.append(cv2.imread(img));
  train_labels.append(4)
print('Loaded plastic images...')
for img in glob.glob(r'/gdrive/My Drive/CoLab/TrashTraining/trash/*.jpg'): 
  train_images.append(Image.open(img));
  #train_images.append(cv2.imread(img));
  train_labels.append(5)
print('Loaded trash images...')
print('Loaded all images!')

newtrain_images = []
for i in range(2527):
  #newtrain_images.append(cv2.resize(train_images[i],(64,48),interpolation=cv2.INTER_AREA));
  train_images[i] = train_images[i].resize((48,64));
  newtrain_images.append(np.array(train_images[i].getdata()).reshape(train_images[i].size[0], train_images[i].size[1], 3));

train_images = np.array(newtrain_images) 
train_images = train_images/255.0 #scale images for 0-1 coefficients
train_labels = np.array(train_labels)

train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.20, shuffle=True)
test_images, validation_images, test_labels, validation_labels = train_test_split(test_images, test_labels, test_size=0.5)

#train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.20, shuffle=True)
#test_images, validation_images, test_labels, validation_labels = train_test_split(test_images, test_labels, test_size=0.5)
pixelRows = train_images[0].shape[0] #pixel dim in y
pixelCols = train_images[0].shape[1] #pixel dim in x
class_names = ['carboard','glass','metal','paper','plastic','trash']
n = 36 #make this a perfect square for nice visuals
plt.figure(figsize=(10,10)) #change this if they look weird
for i in range(n):
  plt.subplot(int(np.sqrt([n])),int(np.sqrt([n])),i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(train_images[i])
  plt.xlabel(class_names[train_labels[i]])
plt.show()


In [0]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(
    150,
    (6,8),
    input_shape=(48, 64, 3),
    strides=1,
    padding='SAME',
    dilation_rate=10,
    activation='selu', #selu
    use_bias=True,
    kernel_initializer='glorot_uniform', #glorot_uniform
    bias_initializer='glorot_uniform', #glorot_uniform 
    kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01),
    bias_regularizer=None,
    activity_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01), #keras.regularizers.l1_l2(l1=0.01, l2=0.01) better but takes longer, keras.regularizers.l2(0.) works about the same
    kernel_constraint='UnitNorm',
    bias_constraint='MaxNorm'))
model.add(keras.layers.MaxPooling2D((3, 4)))
model.add(keras.layers.Conv2D(
    150,
    (6,8),
    input_shape=(48, 64, 3),
    strides=1,
    padding='SAME',
    dilation_rate=10,
    activation='selu', #selu
    use_bias=True,
    kernel_initializer='glorot_uniform', #glorot_uniform
    bias_initializer='glorot_uniform', #glorot_uniform 
    kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01),
    bias_regularizer=None,
    activity_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01), #better but takes longer, keras.regularizers.l2(0.) #works about the same
    #keras.regularizers.l1_l2(l1=0.01, l2=0.01), 
    kernel_constraint='UnitNorm',
    bias_constraint='MaxNorm'))
model.add(keras.layers.MaxPooling2D((1, 1)))
model.add(keras.layers.Conv2D(8,(6,8)))
model.add(keras.layers.MaxPooling2D((1, 1)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(8, activation='softmax'))
model.compile(optimizer=keras.optimizers.Adadelta(learning_rate=0.1),loss='sparse_categorical_crossentropy',metrics=['accuracy']) #play with hinge loss after everthing else is set up
model.fit(
    x=train_images,
    y=train_labels,
    batch_size=32,
    epochs=20,
    verbose=1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto', baseline=None, restore_best_weights=False)],
    validation_data=(validation_images,validation_labels),
    shuffle=True,
    class_weight=None, #don't need unless insufficient data for a given class
    sample_weight=None, #use once good training has been done to speed up processing
    initial_epoch=0, #use if resuming earlier training run
    steps_per_epoch=None,
    validation_steps=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=True)

#check with test imgs
from progressbar import ProgressBar
prdct = []
confidence = []
pbar = ProgressBar()
for i in pbar(range(len(test_images))):
  prdct.append(model.predict(np.expand_dims(test_images[i],axis=0)))
  confidence.append(np.max(prdct[i]))
print('{0:.3f}%'.format(100*np.mean(confidence)))
print('{0:.3f}'.format(100*np.std(confidence)))
#print(confusion_matrix(test_labels,np.argmax(prdct,axis=2)))
prdctnew = []
for i in range(len(test_labels)):
  prdctnew.append(np.argmax(prdct[i], axis=1)[0])
print(f1_score(test_labels,prdctnew,average=None))


In [0]:
print(confusion_matrix(test_labels,np.argmax(prdct,axis=2)))

In [0]:
model.summary()

In [0]:
#manually check accuracy of training
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(3))
  plt.yticks([])
  thisplot = plt.bar(range(6), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

i=10
test_img = np.array([test_images[i]]) #img we want to test
ans = np.array([test_labels[i]]) #0-5 what it should output
prdct=model.predict(test_img)
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(0,prdct[0],ans,test_img)
plt.subplot(1,2,2)
plot_value_array(0,prdct[0],ans) 
_ = plt.xticks(range(6),class_names,rotation=45)
_ = plt.ylabel('Probability')
plt.show()

In [0]:
#test image from drive
#test_img = np.array([cv2.imread(r'/gdrive/My Drive/CoLab/Inputs/input6.jpg',0)])/255
test_img = train_images[1]
test_img = np.expand_dims(test_img,axis=0)
prdct=model.predict(test_img)
print(test_img.shape) #leave this cuz for some reason it fixes
print(prdct)
plt.figure(figsize=(3,3))
plt.imshow(test_img[0])
np.argmax(prdct)
plt.xticks([])
plt.yticks([])
plt.xlabel("{} {:2.0f}%".format(class_names[np.argmax(prdct)],100*np.max(prdct[0])));
